In [ ]:
// 1- Setting temporal periods
var periods = [
  {label: '2017_AJ', start: '2017-04-01', end: '2017-07-31'},
  {label: '2018_AJ', start: '2018-04-01', end: '2018-07-31'},
  {label: '2019_AJ', start: '2019-04-01', end: '2019-07-31'}
  ];

// 2- Function to calculate indices
function addIndices(image) {
  var ndvi = image.normalizedDifference(['B8','B4']).rename('ndvi');
  var evi = image.expression(
    '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
      'NIR': image.select('B8').multiply(0.0001),
      'RED': image.select('B4').multiply(0.0001),
      'BLUE': image.select('B2').multiply(0.0001)
      }
      ).rename('evi');
  var savi = image.expression(
    '((NIR - RED) / (NIR + RED + 0.5)) * (1.5)', {
      'NIR': image.select('B8').multiply(0.0001),
      'RED': image.select('B4').multiply(0.0001)
      }
      ).rename('savi');
  var ndwi = image.normalizedDifference(['B8','B11']).rename('ndwi');
  var ndre = image.normalizedDifference(['B8','B5']).rename('ndre');
  var msavi = image.expression(
    '(2 * NIR + 1 - sqrt((2 * NIR + 1) ** 2 - 8 * (NIR - RED))) / 2', {
      'NIR': image.select('B8'),
      'RED': image.select('B4')
    }).rename('MSAVI');
  return image.addBands([ndvi, evi, savi, ndwi, ndre, msavi]);
}

// 4. Loop over the periods
var allStats = periods.map(function(p){
  var collection = ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
  .filterDate(p.start, p.end)
  .filterBounds(buffers200)
  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
  .map(addIndices);

  var composite = collection.median().set('period' , p.label);


// 5. Zonal statistics for all buffers

  var zonalstats = composite.reduceRegions({
    collection: buffers200,
    reducer: ee.Reducer.mean(),
    scale:10
  }).map(function(f) {
    return f.set('period', p.label);
  });

    return zonalstats;
});

// 6. Merging all stats for export
var mergedStats = ee.FeatureCollection(allStats).flatten();

Export.table.toDrive({
  collection: mergedStats,
  description: 'zonalstats_buffers200',
  fileFormat: 'csv'
  });

Export.table.toDrive({
  collection: mergedStats,
  description: 'zonalstats_buffers200',
  fileFormat: 'geojson'
  });
